

# Scotiabank Quarterly Reports: Instant Analysis and Translation Powered by Custom Made Chatbot

Source: https://www.scotiabank.com/ca/en/about/investors-shareholders/financial-result.html

## Team Alpha

* Raemil Corniel
* Uma Maheshwari
* Sonal Parmar
* Jharana Adhikari
* Aparna Suresh1


Table of Content
0.  Install libraries needed to run the notebook
1.  Load Libraries
2.  Class and functions
3.  Load data
4.  Analysis data (before cleaning)
5.  Clean data
6.  Exploratory Data Analysis
7.  Modeling
8.  Metrics Evaluation
9.  Conclusion






### 0. Install libraries needed to run the notebook

In [1]:
pip install tabula-py pypdf PdfReader pdfminer pdfminer.six jpype1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.0 MB/s eta 0:00:00
  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140076 sha256=ae87cf35c91bfbf45165a18806c57303c21a47857e05fbee3308e73b7a1d8a1f
  Stored in directory: /root/.cache/pip/wheels/4e/c1/68/f7bd0a8f514661f76b5cbe3b5f76e0033d79f1296012cbbf72
Successfully built pdfminer


### 1. Load Libraries

In [2]:
import pandas as pd
import re
import math

from pypdf import PdfReader
from pdfminer.converter import TextConverter
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.layout import LAParams
from pdfminer.high_level import extract_text

import io
import tabula
import jpype

from bs4 import BeautifulSoup

In [3]:
#show all columns in a df
pd.set_option('display.max_columns', None)
#show full row info
pd.set_option('display.max_colwidth', None)

In [4]:
print("JPype version:", jpype.__version__)


JPype version: 1.5.0


### 2. Classes and Function
#### This is to avoid code repetition and for code clarity and reusability  

In [5]:
#load file, given a pdf file path
def load_file(pdf):
    #load the file
    document = PdfReader(pdf , 'rb')

    counter=0
    textList = []

    # only read first 11 pages the last 3 pages are not relevant (Forward-looking statements, Shareholders Information)
    while counter < (len(document.pages) -3):
        pageObj = document.pages[counter]

        textList.append(pageObj.extract_text())
        counter += 1

    #return a list with the pages of the document, this list does not contain tables only textual info
    return textList


In [6]:
#clean text from html tags, \n, extra spaces, urls, double spaces , lower case
def clean_text(fileText):

    #clean file text
    textCleannedList = []
    for textToProccess in fileText:
        #lower case
        textToProccess = textToProccess.lower()

        #remove all tags from html
        textToProccess= BeautifulSoup(textToProccess, 'html.parser').get_text()

        #remove newlines
        textToProccess= textToProccess.replace('\n',' ')

        #remove spaces
        textToProccess = textToProccess.strip()

        #remove urls
        patterns = re.compile(r'https?://\S+|www\.\S+')
        textToProccess = patterns.sub(' ', textToProccess)

        #remove double space
        textToProccess =re.sub("\s\s+" , " ", textToProccess)

        textCleannedList.append(textToProccess)

    return textCleannedList

In [7]:
#get text from specific part of the document
#given a list of pages, start word, end word , and a tag for identifying type of information extracted
def extract_text_between_words_across_pages(text_pages, start_word, end_word, tag):
    extracted_texts = []
    #join all pages
    combined_text = "\n".join(text_pages)
    pattern = re.compile(re.escape(start_word) + '(.*?)' + re.escape(end_word), re.S)

    for match in pattern.finditer(combined_text):
        start_page, end_page = None, None
        start_index, end_index = match.start(), match.end()

        cumulative_length = 0
        for i, page in enumerate(text_pages):
            page_length = len(page)
            if start_page is None and start_index < cumulative_length + page_length:
                start_page = i + 1
            if end_page is None and end_index <= cumulative_length + page_length:
                end_page = i + 1
            cumulative_length += page_length + len("\n")
            if start_page is not None and end_page is not None:
                break

        extracted_texts.append({
            'start_page': start_page,
            'end_page': end_page,
            'start_word': start_word,
            'end_word': end_word,
            'tag':tag,
            'extracted_text': start_word +' '+ match.group(1).strip()
        })
    return extracted_texts

In [8]:
#given a list of pages, and  list of set of words (start, end) return a list of findings
def process_files_in_folder(text_pages, start_end_pairs, tag):
    all_extracted_texts = []

    for start_word, end_word in start_end_pairs:
        extracted_texts = extract_text_between_words_across_pages(text_pages, start_word, end_word,tag)
        all_extracted_texts.extend(extracted_texts)

    return all_extracted_texts


In [364]:

# Function to add dollar sign to numeric values
def add_dollar_sign(value):
    if isinstance(value, str) and value.replace(',', '').isdigit():
        return f'${value}'

    if isinstance(value, str) and '-' in value:
        return f'${value}'

    return value


# Define the function to convert values
def add_dollar_negative_sign(value):
    if isinstance(value, str):
        # Check if the value is in parentheses
        match = re.match(r'\(([\d,]+)\)', value)
        if match:
            # Extract the number, convert to negative, and format with dollar sign
            number = match.group(1).replace(',', '')
            return f'${-int(number):,}'
    return value




# Function to create new column names
def add_headers(df,numColJoin):
    new_columns = []
    for col in df.columns:
       # Join the first 3 rows for this column
       if 'Unnamed' in col:
          joined_name = ' '.join(df[col].iloc[:numColJoin].dropna().astype(str))
       else:
          joined_name =  col + ' ' + ' '.join(df[col].iloc[:numColJoin].dropna().astype(str))

       new_columns.append(joined_name)

    return new_columns


# Function to clean column names from (number) and .number
def remove_number_header(name):
    # Remove patterns like (number) and .number
    cleaned_name = re.sub(r'\(\d+\)', '', name)  # Remove (number)
    cleaned_name = re.sub(r'\.\d+', '', cleaned_name)  # Remove .number
    return cleaned_name.strip()



#if any row has double text then join them
def join_row_double_text(df):
    rows = []

    for i in range(len(df)):
        if df.iloc[i, 0] == 'Adjustments:':
          continue


        if i > 0 and df.iloc[i-1][1:].isna().all():

            # Previous row is all NaN, current row contains values
            merged_name = df.iloc[i-1, 0] + ' ' + df.iloc[i, 0] if not pd.isna(df.iloc[i, 0]) else df.iloc[i-1, 0]
            # Create a new row with merged column name and current row's values
            new_row = [merged_name] + list(df.iloc[i, 1:])
            rows.append(new_row)
        else:
            # Normal row, just add it
            rows.append(df.iloc[i].tolist())

    # Create a new DataFrame from the rows
    result_df = pd.DataFrame(rows, columns=df.columns)

    return result_df


# Function to split DataFrame based on specific text
def split_dataframe_on_text(df, text, check_columns):
    split_indices = []
    tables = []

    # Track the start of the current table
    start_idx = 0

    for i in range(len(df)):
        row = df.iloc[i]
        # Check if column 0 is NaN or null
        if pd.isna(row[0]):
            # Check other specified columns for the text
            found = any(text in str(row[col]) for col in check_columns)
        else:
            # Check if the text is in column 0
            found = text in str(row[0])

        # If the text is found, add the index to the split_indices list
        if found:
            # Check if this row contains the specified text
            if i > start_idx:
                split_indices.append(i)
                tables.append(df.iloc[start_idx:i])
                start_idx = i
    # Append the last segment
    if start_idx < len(df):
        tables.append(df.iloc[start_idx:])

    return tables


def split_dataframe_on_text_column(df, text, check_column):
    split_indices = []
    tables = []

    # Track the start of the current table
    start_idx = 0

    for i in range(len(df)):
        row = df.iloc[i]
        # Check if column 0 is NaN or null
        found = text in str(row[check_column])

        # If the text is found, add the index to the split_indices list
        if found:
            # Check if this row contains the specified text
            if i > start_idx:
                split_indices.append(i)
                tables.append(df.iloc[start_idx:i])
                start_idx = i
    # Append the last segment
    if start_idx < len(df):
        tables.append(df.iloc[start_idx:])

    return tables


# Function to split rows with multiple values
def split_rows(df):
    new_rows = []
    for index, row in df.iterrows():
        # Check for rows with space-separated values
        for col in df.columns:
            if isinstance(row[col], str) and ' ' in row[col]:
                # Split the cell content by space
                split_values = row[col].split()
                # If there are more than 2 values, handle them
                if len(split_values) > 1:
                    # Create new rows based on split values
                    for i in range(len(split_values)):
                        new_row = row.copy()
                        new_row[col] = split_values[i]
                        new_rows.append(new_row)
                else:
                    new_rows.append(row)
            else:
                new_rows.append(row)

    # Create a new DataFrame with the expanded rows
    return pd.DataFrame(new_rows)

def remove_dollar_sign(df):
    # Define a function to process each cell
    def remove_dollar_sign_from_cell(cell):
        if isinstance(cell, str) and cell.endswith('$'):
            # Remove the dollar sign from the end of the string
            return cell.rstrip('$')
        return cell

    # Apply the function to each cell in the DataFrame
    return df.applymap(remove_dollar_sign_from_cell)





### 3. Load data



> In this step the data will be load twice, once using the library pypdf for gather all the text of the report, and using tabula, this library will help us load all the tables of the report in a more propper format.



In [9]:
# Replace 'your_pdf_file.pdf' with the path to your PDF file
pdf_path = "Q224.pdf"

#get file text , provide pdf path
file_text = load_file(pdf_path)




# Extract all tables from the PDF
tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True)



#indicate the quater and year
quater= 'second'
year = '2024'

### 4. Analysis of the data (before cleaning)

In [10]:
#len of the document
print('Total pages', len(file_text))


#words per page
for index , page in enumerate(file_text):
    words_per_page = len(page.split())
    print(f'Total words page #{index} :{ words_per_page}')

#amount of words in the document
total_words = sum(len(page.split()) for page in file_text)
print('\nTotal Words:', total_words)



print(f'Total tables available: {len(tables)}')


Total pages 11
Total words page #0 :663
Total words page #1 :726
Total words page #2 :959
Total words page #3 :1042
Total words page #4 :252
Total words page #5 :379
Total words page #6 :574
Total words page #7 :644
Total words page #8 :634
Total words page #9 :739
Total words page #10 :761

Total Words: 7373
Total tables available: 6




> The financial Quarterly Release report of Scotiabank has ~14 pages but, only the frist 11 has been taken for analysis, since the last pages do not provide useful information



*   Total pages 11
*   Total Words: 7373
*   Tables: 6

> Actually the report has 10 tables, which means that 4 tables are not read correctly, in the next steps this will be fixed
















### 5. Clean the data




> In this step the data obtained from the pdf will be cleaned and adapted for proper use, with that said the next steps will be applied:

* Remove HTML tags
* Lower case text
* remove \n
* remove url
* remove extra spaces



> Similar steps will be applied again for extracting and cleaning the tables





In [11]:
#show lenght of document
print('Document total pages:',len(file_text))
#by showing second page we expect to see a tables, in text format
print ('Second page: ')
file_text[1]


Document total pages: 11
Second page: 


' \n                     \n            \n                \n            \n           \n                \n          \n                  \n                \n                 \n          \n                      \n                \n         \n           \n              \n          \n         \n \n                                 \n          \n \n \n     \n \n                    \n                   \n                         \n                          \n           \n \n    \n   \n      \n                        \n                \n      \n                         \n             \n        \n                       \n                       \n   \n      \n                      \n                         \n                \n      \n                       \n                       \n                        \n      \n        \n                       \n                        \n                       \n Financial  Highlights  \nReported Results For the three months ended For the six months ended \n

In [12]:
#return a list with text cleanned remove   html tags, \n, extra spaces, urls, double spaces , lower case
text_cleanned = clean_text(file_text)

In [13]:
text_cleanned[1] #show frist page after cleanned
#look that the table of financial highlights is visible, we will remove this table
#since it will be loaded later in a df with the right format

'financial highlights reported results for the three months ended for the six months ended (unaudited) ($ millions) april 30 2024(1) january 31 2024(1) april 30 2023(1) april 30 2024(1) april 30 2023(1) operating results net interest income $ 4,694 $ 4,773 $ 4,460 $ 9,467 $ 9,023 non -interest income 3,653 3,660 3,453 7,313 6,852 total revenue $ 8,347 $ 8,433 $ 7,913 $ 16,780 $ 15,875 provision for credit losses 1,007 962 709 1,969 1,347 non -interest expenses 4,711 4,739 4,574 9,450 9,035 income tax expense 537 533 484 1,070 1,589 net income $ 2,092 $ 2,199 $ 2,146 $ 4,291 $ 3,904 net income attributable to non-controlling interests in subsidiaries 26 25 24 51 61 net income attributable to equity holders of the bank $ 2,066 $ 2,174 $ 2,122 $ 4,240 $ 3,843 preferred shareholders and other equity instrument holders 123 108 104 231 205 common shareholders $ 1,943 $ 2,066 $ 2,018 $ 4,009 $ 3,638 earnings per common share (in dollars) basic $ 1.59 $ 1.70 $ 1.69 $ 3.29 $ 3.05 diluted $ 1.57



> For better understanding of the document, it will be separated into chunks of information, which will give us information classified and organized according to the report, this way we can easily search for key topics like, global banking and markets.



> A list of sets will be created indicating the start and end of the chunks desire







In [14]:
#tables we will removed from text since it will imported using tabula

#create a list of set with keyword that indicates the begining and end of a part of
#information of in the document.


summary_set = [
    (quater + ' quarter ' + year+' earnings release' , quater + ' quarter ' + year+' highlights on a reported basis'),
    (quater + ' quarter ' + year+' highlights on a reported basis' , quater + ' quarter ' + year+' highlights on an adjusted basis'),
    (quater + ' quarter ' + year+' highlights on an adjusted basis' , 'toronto,'),
    ('toronto,' , 'financial highlights'),
]


financial_ighlights  = [
    ('reported results ', 'business segment review'),
]

business_segment_review = [
    ('business segment review', 'credit risk'),
    ('canadian banking', 'international banking'),
    ('international banking', 'financial performance on a constant dollar basis'),
    ('financial performance on a constant dollar basis','global wealth management'),
    ('global wealth management', 'global banking and markets'),
    ('global banking and markets', 'other'),
    ('other', 'credit risk'),
]

credit_risk = [
    ('credit risk','capital ratios'),
    ('provision for credit losses', 'allowance for credit losses'),
    ('allowance for credit losses', 'impaired loans'),
    ('impaired loans','capital ratios'),
]

capital_ratios = [
    ('capital ratios','non-gaap measures')
]

non_gaap_measures = [
    ('non-gaap measures','reconciliation of reported and adjusted results and diluted earnings per share'),
    ('non-gaap measures','adjusted results and diluted earnings per share'),
    ('adjusted results and diluted earnings per share','reconciliation of reported and adjusted results and diluted earnings per share'),


]

reconciliation= [
    ('reconciliation of reported and adjusted results and diluted earnings per share','reconciliation of reported and adjusted results by business line'),
    ('reconciliation of reported and adjusted results by business line','reconciliation of International Banking’sreported, adjusted and constant dollar results'),
    ('reconciliation of International Banking’sreported, adjusted and constant dollar results', 'return on equity'),
]

return_on_equity = [
        ('return on equity','return on equity by operating segment'),
        ('return on equity by operating segment','forward-looking statements'),

]



In [15]:
#information used for report: list of dictionaries

summary_part =  process_files_in_folder(text_cleanned, summary_set, 'summary')
financial_ighlights_part =  process_files_in_folder(text_cleanned, financial_ighlights, 'financial_ighlights')
business_segment_review_part =  process_files_in_folder(text_cleanned, business_segment_review, 'business_segment_review')
credit_risk_part =  process_files_in_folder(text_cleanned, credit_risk, 'credit_risk')
capital_ratios_part =  process_files_in_folder(text_cleanned, capital_ratios, 'capital_ratios')
non_gaap_measures_part =  process_files_in_folder(text_cleanned, non_gaap_measures, 'non-gaap-measures')
reconciliation_part =  process_files_in_folder(text_cleanned, reconciliation, 'reconciliation')
return_on_equity_part =  process_files_in_folder(text_cleanned, return_on_equity, 'return_on_equity')





> For ease of use, the chunks have been converted to a dataframe.




In [16]:

# Convert each list of dictionaries to a DataFrame
summary_df = pd.DataFrame(summary_part)
financial_highlights_df = pd.DataFrame(financial_ighlights_part)
business_segment_review_df = pd.DataFrame(business_segment_review_part)
credit_risk_df = pd.DataFrame(credit_risk_part)
capital_ratios_df = pd.DataFrame(capital_ratios_part)
non_gaap_measures_df = pd.DataFrame(non_gaap_measures_part)
reconciliation_df = pd.DataFrame(reconciliation_part)
return_on_equity_df = pd.DataFrame(return_on_equity_part)

# Combine all DataFrames into one
finnacial_release_df = pd.concat([
    summary_df,
    financial_highlights_df,
    business_segment_review_df,
    credit_risk_df,
    capital_ratios_df,
    non_gaap_measures_df,
    reconciliation_df,
    return_on_equity_df
], ignore_index=True)


print('Shape',finnacial_release_df.shape)

Shape (51, 6)


In [17]:
finnacial_release_df.head() #get first 5 rows

,start_page,end_page,start_word,end_word,tag,extracted_text
0,1,1,second quarter 2024 earnings release,second quarter 2024 highlights on a reported basis,summary,"second quarter 2024 earnings release scotiabank reports second quarter results all amounts are in canadian dollars and are based on our unaudited interim condensed consolidated financial statements for the quarter ended april 30, 2024 and related notes prepared in accordance with international financial reporting standards (ifrs) as issued by the international accounting standards board (iasb), unless otherwise noted. our complete second quarter 2024 report to shareholders, including our unaudited interim financial statements for the period ended april 30, 2024, can also be found on the sedar+ website at and on the edgar section of the sec’s website at . supplementary financial information is also available, together with the second quarter 2024 report to shareholders on the investor relations page at ."
1,1,1,second quarter 2024 highlights on a reported basis,second quarter 2024 highlights on an adjusted basis,summary,"second quarter 2024 highlights on a reported basis (versus q2 2023) • net income of $2,092 million, compared to $2,146 million • earnings per share (diluted) of $1.57, compared to $1.68 • return on equity(2) of 11.2%, compared to 12.2%"
2,1,1,second quarter 2024 highlights on an adjusted basis,"toronto,",summary,"second quarter 2024 highlights on an adjusted basis (1) (versus q2 2023) • net income of $2,105 million, compared to $2,161 million • earnings per share (diluted) of $1.58, compared to $1.69 • return on equity of 11.3%, compared to 12.3%"
3,1,2,"toronto,",financial highlights,summary,"toronto, may 28, 2024 —the bank of nova scotia (“scotiabank”) (tsx: bns; nyse: bns) reported second quarter net income of $2,092 million compared to $2,146 million in the same period last year. diluted earnings per share (eps) were $1.57, compared to $1.68 in the same period a year ago. adjusted net income(1) for the second quarter was $2,105 million and adjusted diluted eps(1) was $1.58 , down from $1.69 last year. adjusted return on equity(1) was 11.3% compared to 12.3% a year ago. “the bank delivered solid results this quarter against a backdrop of ongoing macroeconomic uncertainty, reporting positive operating leverage driven by revenue growth and continued expense discipline. we are executing on our commitment to balanced growth as our deposit momentum continues, while maintaining strong capital and liquidity metrics,” said scott thomson, president and ceo of scotiabank. “iam prou d to see scotiabankers across our global footprint rallying behind our new strategy and coming together to drive our key strategic initiatives forward.” canadian banking delivered adjusted earnings(1) of $1 billion this quarter. solid revenue growth outpaced expense growth resulting in another quarter of positive operating leverage, while provision for credit losses increased compared to the prior year. in addition, deposit growth, a key component of the refreshed strategy, was up 7% year -over -year . international banking generated adjusted earnings(1) of $701 million. revenue growth driven by strong margin expansion , disciplined expense and capital management, were offset by higher provision for credit losses. adjusted return on equity(1) was 14.5%, a 120 basis point improvement from last year . global wealth management adjusted earnings(1) were $389 million, up 8% year over year. assets under management(2) of $349 billion increased by 6% resulting in strong revenue growth , partly offset by investments to support long -term business growth . global banking and markets reported earnings of $428 million, up 7% compared to the prior year. results were supported by higher fee-based revenue and lower provision for credit losses . the bank reported a common equity tier 1 (cet1) capital ratio(3) of 13.2%, up from 12.3% last year . (1) refer to non -gaap measures section starting on 

In [18]:
finnacial_release_df.tail(10) # get last 10 rows


start_page  end_page  \
41           1         4   
42           4         5   
43           1         4   
44           4         4   
45           4         5   
46           1         6   
47           6         7   
48           6         7   
49           7         8   
50           1        11   

                                                                        start_word  \
41                                                                           other   
42                                                                     credit risk   
43                                                     provision for credit losses   
44                                                     allowance for credit losses   
45                                                                  impaired loans   
46                                                                  capital ratios   
47                                                               non-gaap measures   
48                                                               non-gaap measures   
49  reconciliation of reported and adjusted results and diluted earnings per share   
50                                                                return on equity   

                                                                          end_word  \
41                                                                     credit risk   
42                                                                  capital ratios   
43                                                     allowance for credit losses   
44                                                                  impaired loans   
45                                                                  capital ratios   
46                                                               non-gaap measures   
47  reconciliation of reported and adjusted results and diluted earnings per share   
48                                 adjusted results and diluted earnings per share   
49                reconciliation of reported and adjusted results by business line   
50                                           return on equity by operating segment   

                        tag  \
41  business_segment_review   
42              credit_risk   
43              credit_risk   
44              credit_risk   
45              credit_risk   
46           capital_ratios   
47        non-gaap-measures   
48        non-gaap-measures   
49           reconciliation   
50         return_on_equity   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [19]:
#rows with no meaning
finnacial_release_df[finnacial_release_df['extracted_text'].apply(len) <100]

,start_page,end_page,start_word,end_word,tag,extracted_text
11,6,6,canadian banking,international banking,business_segment_review,"canadian banking ,"
12,8,8,canadian banking,international banking,business_segment_review,canadian banking (2)
13,8,8,canadian banking,international banking,business_segment_review,canadian banking (2)
14,9,9,canadian banking,international banking,business_segment_review,canadian banking (2)
15,9,9,canadian banking,international banking,business_segment_review,canadian banking (2)
16,10,10,canadian banking,international banking,business_segment_review,canadian banking (2)
17,11,11,canadian banking,international banking,business_segment_review,canadian banking (1)
18,11,11,canadian banking,international banking,business_segment_review,canadian banking (1)
25,8,8,global wealth management,global banking and markets,business_segment_review,global wealth management
26,9,9,global wealth management,global banking and markets,business_segment_review,global wealth management


In [20]:
#drop columns if extracted text is less than 100 letters
finnacial_release_df.drop(finnacial_release_df[finnacial_release_df['extracted_text'].apply(len) <100].index, inplace=True)

In [21]:

print('Shape',finnacial_release_df.shape)
finnacial_release_df.head()

Shape (30, 6)


,start_page,end_page,start_word,end_word,tag,extracted_text
0,1,1,second quarter 2024 earnings release,second quarter 2024 highlights on a reported basis,summary,"second quarter 2024 earnings release scotiabank reports second quarter results all amounts are in canadian dollars and are based on our unaudited interim condensed consolidated financial statements for the quarter ended april 30, 2024 and related notes prepared in accordance with international financial reporting standards (ifrs) as issued by the international accounting standards board (iasb), unless otherwise noted. our complete second quarter 2024 report to shareholders, including our unaudited interim financial statements for the period ended april 30, 2024, can also be found on the sedar+ website at and on the edgar section of the sec’s website at . supplementary financial information is also available, together with the second quarter 2024 report to shareholders on the investor relations page at ."
1,1,1,second quarter 2024 highlights on a reported basis,second quarter 2024 highlights on an adjusted basis,summary,"second quarter 2024 highlights on a reported basis (versus q2 2023) • net income of $2,092 million, compared to $2,146 million • earnings per share (diluted) of $1.57, compared to $1.68 • return on equity(2) of 11.2%, compared to 12.2%"
2,1,1,second quarter 2024 highlights on an adjusted basis,"toronto,",summary,"second quarter 2024 highlights on an adjusted basis (1) (versus q2 2023) • net income of $2,105 million, compared to $2,161 million • earnings per share (diluted) of $1.58, compared to $1.69 • return on equity of 11.3%, compared to 12.3%"
3,1,2,"toronto,",financial highlights,summary,"toronto, may 28, 2024 —the bank of nova scotia (“scotiabank”) (tsx: bns; nyse: bns) reported second quarter net income of $2,092 million compared to $2,146 million in the same period last year. diluted earnings per share (eps) were $1.57, compared to $1.68 in the same period a year ago. adjusted net income(1) for the second quarter was $2,105 million and adjusted diluted eps(1) was $1.58 , down from $1.69 last year. adjusted return on equity(1) was 11.3% compared to 12.3% a year ago. “the bank delivered solid results this quarter against a backdrop of ongoing macroeconomic uncertainty, reporting positive operating leverage driven by revenue growth and continued expense discipline. we are executing on our commitment to balanced growth as our deposit momentum continues, while maintaining strong capital and liquidity metrics,” said scott thomson, president and ceo of scotiabank. “iam prou d to see scotiabankers across our global footprint rallying behind our new strategy and coming together to drive our key strategic initiatives forward.” canadian banking delivered adjusted earnings(1) of $1 billion this quarter. solid revenue growth outpaced expense growth resulting in another quarter of positive operating leverage, while provision for credit losses increased compared to the prior year. in addition, deposit growth, a key component of the refreshed strategy, was up 7% year -over -year . international banking generated adjusted earnings(1) of $701 million. revenue growth driven by strong margin expansion , disciplined expense and capital management, were offset by higher provision for credit losses. adjusted return on equity(1) was 14.5%, a 120 basis point improvement from last year . global wealth management adjusted earnings(1) were $389 million, up 8% year over year. assets under management(2) of $349 billion increased by 6% resulting in strong revenue growth , partly offset by investments to support long -term business growth . global banking and markets reported earnings of $428 million, up 7% compared to the prior year. results were supported by higher fee-based revenue and lower provision for credit losses . the bank reported a common equity tier 1 (cet1) capital ratio(3) of 13.2%, up from 12.3% last year . (1) refer to non -gaap measures section starting on 

In [556]:
#save it for future use

finnacial_release_df.to_csv('finnacial_release_df.csv', index=False)



> After checking all the data extracted and noticing some rows had repeated information with no meaning at all, they were removed, and a df was obtained with only 30 rows and 6 features that will be used for training the models.


> Now it's time to load the tables in a format more readable like a table, for that tabula will be used, all the tables will be extracted, cleaned and given the proper format





In [23]:
#check tables format

#first table
tables[0]

#frist table looks in a good shape, some negatives values needs to be
#modifed to look more like real values ex. negatives, nan

,Unnamed: 0,Unnamed: 1,April 30,Unnamed: 2,January 31,Unnamed: 3,April 30.1,Unnamed: 4,April 30.2,Unnamed: 5,April 30.3
0,(Unaudited) ($ millions),NaN,2024(1),NaN,2024(1),NaN,2023(1),NaN,2024(1),NaN,2023(1)
1,Operating results,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Net interest income,$,"4,694",$,"4,773",$,"4,460",$,"9,467",$,"9,023"
3,Non-interest income,NaN,"3,653",NaN,"3,660",NaN,"3,453",NaN,"7,313",NaN,"6,852"
4,Total revenue,$,"8,347",$,"8,433",$,"7,913",$,"16,780",$,"15,875"
5,Provision for credit losses,NaN,"1,007",NaN,962,NaN,709,NaN,"1,969",NaN,"1,347"
6,Non-interest expenses,NaN,"4,711",NaN,"4,739",NaN,"4,574",NaN,"9,450",NaN,"9,035"
7,Income tax expense,NaN,537,NaN,533,NaN,484,NaN,"1,070",NaN,"1,589"
8,Net income,$,"2,092",$,"2,199",$,"2,146",$,"4,291",$,"3,904"
9,Net income attributable to non-controlling interests in subsidiaries,NaN,26,NaN,25,NaN,24,NaN,51,NaN,61


In [134]:
#let's check the last table
tables[len(tables)-1]

#this table needs more work done since this table has at least 2 tables merged together
#the library was not able to capture it correctly

,"($ millions) January 31, 2024(1)","April 30, 2023(1)","April 30, 2023(1).1"
0,Constant,Constant,Constant
1,Foreign dollar,Foreign dollar,Foreign dollar
2,(Taxable equivalent basis) Adjusted exchange a...,Adjusted exchange adjusted,Adjusted exchange adjusted
3,"Net interest income $ 2,246 $ 19 $ 2,227","$ 1,999 $ 8 $ 1,991","$ 3,891 $ (82) $ 3,973"
4,Non-interest income 857 6 851,743 (88) 831,"1,535 (163) 1,698"
5,"Total revenue 3,103 25 3,078","2,742 (80) 2,822","5,426 (245) 5,671"
6,Provision for credit losses 574 6 568,436 (3) 439,840 (27) 867
7,"Non-interest expenses 1,563 2 1,561","1,467 (24) 1,491","2,890 (99) 2,989"
8,Income tax expense 192 4 188,174 (10) 184,345 (20) 365
9,Net income $ 774 $ 13 $ 761,$ 665 $ (43) $ 708,"$ 1,351 $ (99) $ 1,450"




> To clean the tables and obtain the proper format, series of steps will be applied.

* add dollar sign to numeric values
* add dollar sign and negative to numeric values if the value is negative
* add header taking into consideration the number of rows that form part of the header
* remove innecesary columns or rows, like NaN or $ only
* join row base on the premises that if the prior column has in its 0 column some text and the other values are NaN, and the actual row is full of text and number then the text of the 0 column should be join as one.
* split the dataframe by rows when specific text is found (this is to give the proper format to each table, since some tables were load together
* remove extra dollar sign found on some numbers at the end



In [413]:
print(f'Total Tables: {len(tables)}\n')


#check the amount of col and row ecach table has before cleaning
for index, table in enumerate(tables):
  print(f'\n Table number: {index}')
  print(f'Table dimmensions: {table.shape}' )



Total Tables: 6


 Table number: 0
Table dimmensions: (16, 11)

 Table number: 1
Table dimmensions: (49, 11)

 Table number: 2
Table dimmensions: (49, 10)

 Table number: 3
Table dimmensions: (49, 12)

 Table number: 4
Table dimmensions: (52, 12)

 Table number: 5
Table dimmensions: (54, 3)




> Since each table has been loaded in diffent structure, is needed to analyze the tables and apply the proper fix to each table, in order to match the report table



In [565]:
#result table
result_table =[]

#iterate tables and apply different cleaning techniques depending on the table
for index, table in enumerate(tables):

  #get table clean
  df = table


  #changes to be apply to table 0 - Financial Highlights Reported Results
  if index == 0:

      # Apply the function to all cells in the DataFrame
      df = df.apply(lambda col: col.map(add_dollar_sign))
      #add $- to negative values
      df = df.apply(lambda col: col.map(add_dollar_negative_sign))

      # Apply the function to get new column names, number of col to take
      df.columns = add_headers(df,1)

      #remove row 0 and 1 the headers were added so is not necessary
      df.drop(index=[0], inplace=True)

      #drop all column that has no name
      df = df.loc[:, df.columns != '']

      # remove column names from (number) and .number
      df.columns = [remove_number_header(col) for col in df.columns]

      # Replace NaN values with empty strings
      df = df.fillna('')

      #add table 0
      result_table.append(df)


  elif index ==1:
      # Apply the function to all cells in the DataFrame
      df = df.apply(lambda col: col.map(add_dollar_sign))
      #add $- to negative values
      df = df.apply(lambda col: col.map(add_dollar_negative_sign))

      # Apply the function to get new column names
      df.columns = add_headers(df,1)
      #remove row 0 and 1 the headers were added so is not necessary
      df.drop(index=[0], inplace=True)

      # remove column names from (number) and .number
      df.columns = [remove_number_header(col) for col in df.columns]
      #remove second column , it does not contain any valuable info
      df.drop(df.columns[1], axis=1, inplace=True)
      # Apply the function to merge columns
      df = join_row_double_text(df)
      # Drop rows where all columns after the first one are NaN
      df.dropna(subset=df.columns[1:], how='all', inplace=True)

      #add table 0
      result_table.append(df)

  elif (index == 2) | (index == 3):

      # Apply the function to all cells in the DataFrame
      df = df.apply(lambda col: col.map(add_dollar_sign))

      #add $- to negative values
      df = df.apply(lambda col: col.map(add_dollar_negative_sign))

      # Apply the function to get new column names
      df.columns = add_headers(df,2)

      #remove row 0 and 1 the headers were added so is not necessary
      df.drop(index=[0, 1], inplace=True)

      # remove column names from (number) and .number
      df.columns = [remove_number_header(col) for col in df.columns]

      #remove second column , it does not contain any valuable info
      df.drop(df.columns[1], axis=1, inplace=True)

      # Apply the function to merge columns
      df = join_row_double_text(df)

      # Drop rows where all columns after the first one are NaN
      df.dropna(subset=df.columns[1:], how='all', inplace=True)

      split_text="For the three months"

      if index == 3 :
        split_text = "For the six months"

      check_columns = [0, 1, 2, 3, 4, 5, 6]  # List of column indices to check

      #add resutls to new table
      new_tables = split_dataframe_on_text(df, split_text, check_columns)

      for table in new_tables:
        check_columns = [0, 1, 2, 3, 4, 5, 6]
        found = any(split_text in str(table.iloc(0)[col]) for col in check_columns)
        if found:
          #remove first 3 rows
          table.drop(table.index[:4], inplace=True)

        result_table.append(table)
  elif index == 4:
      #find row in first column  that start with Reported Results and split the dataset
      # Find indices where the column starts with 'Reported Results'
      # Find the index of rows where the column starts with 'Reported Results'
      new_tables = split_dataframe_on_text_column(table, 'Reported Results', 0)

      for index, tab in enumerate(new_tables):

        if index == 0:

          # Apply the function to get new column names
          tab.columns = add_headers(tab,2)

          #remove row 0 and 1 the headers were added so is not necessary
          tab.drop(index=[0, 1], inplace=True)

          # remove column names from (number) and .number
          tab.columns = [remove_number_header(col) for col in tab.columns]

          # Drop rows where all columns after the first one are NaN
          index_remove = tab[tab['($ millions)'] == 'Adjusted net income attributable to common shareholders'].index

          #remove from index_remove row until end
          tab.drop(index= tab.index[index_remove[0] -1:], inplace=True)

          # Apply the function to merge columns
          tab = join_row_double_text(tab)

          # Drop rows where all columns after the first one are NaN
          tab.dropna(subset=tab.columns[1:], how='all', inplace=True)

          # Drop columns where the column name is an empty string
          tab = tab.loc[:, tab.columns != '']

          #remove dollar sign from the end
          tab = remove_dollar_sign(tab)

          # Apply the function to all cells in the DataFrame
          tab = tab.apply(lambda col: col.map(add_dollar_sign))

          #add $- to negative values
          tab = tab.apply(lambda col: col.map(add_dollar_negative_sign))

          #add table 0
          result_table.append(tab)
        else:
          #create new df
          newDF = pd.DataFrame()

          # Define a regular expression pattern to split based on the presence of a number
          pattern = r'(\D*)\s?(\d[,\d]*)'

          # Extract the text part and the numeric part into separate columns
          newDF[['Reported Results ($ millions)', 'January 31, 2024 Reported']] = tab['Unnamed: 0'].str.extract(pattern, expand=True)

          #drop null values
          newDF.dropna(inplace=True)

          #reset index
          newDF =newDF.reset_index(drop=True)

          #drop index 0
          newDF.drop(index=0, inplace=True)

          #table reset indx
          tab =tab.reset_index(drop=True)

          #drop first 4 rows after secodn column
          tab =tab.iloc[4:,2:]

          # Join all columns into a single column
          tab['combined']= tab.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

          #drop empty rows
          tab.drop(tab[ tab['combined'] == ''].index, inplace=True)

          #remove dollar sign from column combined
          tab['combined'] = tab['combined'].str.replace('$', '')

          #remove double space
          tab['combined'] = tab['combined'].str.replace('  ', ' ')

          # Split the 'Values' column into multiple columns
          df_split = tab['combined'].str.split(' ', expand=True)

          # Rename the columns
          df_split.columns = [f'Value_{i+1}' for i in range(df_split.shape[1])]
          df_split = df_split.reset_index(drop=True)
          newDF = newDF.reset_index(drop=True)

          #join newDF and df_split
          newDF = pd.concat([newDF, df_split], axis=1)

          #add dollar sign
          newDF = newDF.apply(lambda col: col.map(add_dollar_sign))

          #add negatives
          newDF = newDF.apply(lambda col: col.map(add_dollar_negative_sign))

          #rename cols
          newDF.columns = ['Reported Results ($ millions)',
                          'For the three months ended January 31, 2024 Reported',
                          'For the three months ended January 31, 2024 Foreign exchange',
                          'For the three months ended January 31, 2024 Constant dollar',
                          'For the three months ended April 30, 2023 Reported',
                          'For the three months ended April 30, 2023 Foreign exchange',
                          'For the three months ended April 30, 2023 Constant dollar',
                          'For the six months ended April 30, 2023 Reported',
                          'For the six months ended April 30, 2023 Foreign exchange',
                          'For the six months ended April 30, 2023 Constant dollar',
                            ]

          result_table.append(newDF)
  elif index == 5:
      #find row in first column  that start with Reported Results and split the dataset
      # Find indices where the column starts with 'Reported Results'
      # Find the index of rows where the column starts with 'Reported Results'
      new_tables = split_dataframe_on_text_column(df, 'Return on equity by operating segment', 0)

      for index, tab in enumerate(new_tables):

        if index == 0:
          #remove col name
          tab.columns = [f'Value_{i+1}' for i in range(tab.shape[1])]

          # Drop rows where all columns after the first one are NaN
          index_remove = tab[tab['Value_1'].str.contains('Net income attributable to equity holders of the Bank', na = False)].index

          #remove from index_remove row until end
          tab.drop(index= tab.index[index_remove[0] +1 :], inplace=True)
          #drop first 4 rows
          tab =tab.iloc[3:]

          # Join all columns into a single column
          tab['combined']= tab.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

          #remove dollar sign from column combined
          tab['combined'] = tab['combined'].str.replace('$', '')

          #remove double space
          tab['combined'] = tab['combined'].str.replace('  ', ' ')


          #create new df
          newDF = pd.DataFrame()

          # Define a regular expression pattern to split based on the presence of a number
          pattern = r'(\D*)\s?(\d[,\d]*)'

          # Extract the text part and the numeric part into separate columns
          newDF[['Adjusted Results ($ millions)','values']] = tab['combined'].str.extract(pattern, expand=True)
          #remove text from numbers
          newDF['Cleaned'] = tab['combined'].apply(lambda x: re.sub(r'[a-zA-Z]', '', x))

          newDF['Cleaned'] = newDF['Cleaned'].str.replace('-', '')
          newDF['Cleaned'] = newDF['Cleaned'].str.replace('()', '')

          # Split the column based on spaces
          # Convert each split item to a string and remove unwanted characters
          df_split =  newDF['Cleaned'].str.split(r'\s+', expand=True)

          # Drop unnecessary rows if there are empty columns or adjust if needed
          # Rename columns for clarity
          df_split.columns = [f'Value_{i+1}' for i in range(df_split.shape[1])]


          #get only first col
          newDF = newDF[['Adjusted Results ($ millions)']]

          #remove col value! and empty rows
          df_split.drop(df_split.columns[0], axis=1, inplace=True)
          # df_split.dropna(inplace=True)

          df_split = df_split.reset_index(drop=True)
          newDF = newDF.reset_index(drop=True)

          #join newDF and df_split
          newDF = pd.concat([newDF, df_split], axis=1)

          #add dollar sign
          newDF = newDF.apply(lambda col: col.map(add_dollar_sign))

          #add negatives
          newDF = newDF.apply(lambda col: col.map(add_dollar_negative_sign))

          #rename cols
          newDF.columns = ['Adjusted Results ($ millions)',
                          'For the three months ended January 31, 2024 Adjusted',
                          'For the three months ended January 31, 2024 Foreign exchange',
                          'For the three months ended January 31, 2024 Constant dollar adjusted',
                          'For the three months ended April 30, 2023 Adjusted',
                          'For the three months ended April 30, 2023 Foreign exchange',
                          'For the three months ended April 30, 2023 Constant dollar adjusted',
                          'For the six months ended April 30, 2023 Adjusted',
                          'For the six months ended April 30, 2023 Foreign exchange',
                          'For the six months ended April 30, 2023 Constant dollar adjusted',
                            ]

          result_table.append(newDF)

        else:
          tab.reset_index(drop=True, inplace=True)

          #remove col name
          tab.columns = [f'Value_{i+1}' for i in range(tab.shape[1])]

          #drop first 4 rows
          tab =tab.iloc[6:]

          # Join all columns into a single column
          tab['combined']= tab.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

          #remove dollar sign from column combined
          tab['combined'] = tab['combined'].str.replace('$', '')

          #remove double space
          tab['combined'] = tab['combined'].str.replace('  ', ' ')

          #create new df
          newDF = pd.DataFrame()

          #join row title
          strNew = ''
          newList = []
          for a in tab['combined']:
            if len(a) < 30:
              #add to new df
              a = a.replace('(','')
              a = a.replace(')','')
              a = re.sub(r'\d','', a)
              strNew += a + ' '
            else:
              #add to new df
              strNew += a + ' '
              newList.append(strNew)
              strNew = ''


          newDF = pd.DataFrame( newList, columns=['combined'])

          # Define a regular expression pattern to split based on the presence of a number
          pattern = r'(\D*)\s?(\d[,\d]*)'

          # # Extract the text part and the numeric part into separate columns
          newDF[['($ millions)','values']] = newDF['combined'].str.extract(pattern, expand=True)

          #remove text from numbers
          newDF['Cleaned'] = newDF['combined'].apply(lambda x: re.sub(r'[a-zA-Z]', '', x))

          newDF['Cleaned'] = newDF['Cleaned'].str.replace('-', '')
          newDF['Cleaned'] = newDF['Cleaned'].str.replace('()', '')
          newDF['Cleaned'] = newDF['Cleaned'].str.replace('(2)(3)', '')
          newDF['Cleaned'] = newDF['Cleaned'].str.replace('(4)', 'nm(4)')

          # Split the column based on spaces
          # Convert each split item to a string and remove unwanted characters
          df_split =  newDF['Cleaned'].str.split(r'\s+', expand=True)

          # Drop unnecessary rows if there are empty columns or adjust if needed
          # Rename columns for clarity
          df_split.columns = [f'Value_{i+1}' for i in range(df_split.shape[1])]

          #get only first col
          newDF = newDF[['($ millions)']]

          #remove col value! and empty rows
          df_split.drop(df_split.columns[[0,13]], axis=1, inplace=True)
          # df_split.dropna(inplace=True)

          df_split = df_split.reset_index(drop=True)
          newDF = newDF.reset_index(drop=True)

          #join newDF and df_split
          newDF = pd.concat([newDF, df_split], axis=1)

          #add dollar sign
          newDF = newDF.apply(lambda col: col.map( add_dollar_sign))

          #add negatives
          newDF = newDF.apply(lambda col: col.map(add_dollar_negative_sign))

          # #rename cols
          newDF.columns = ['($ millions)',
                          'For the three months ended April 30, 2024 Canadian Banking',
                          'For the three months ended April 30, 2024 International Banking',
                          'For the three months ended April 30, 2024 Global Wealth Management',
                          'For the three months ended April 30, 2024 Global Banking and Markets',
                          'For the three months ended April 30, 2024 Other',
                          'For the three months ended April 30, 2024 Total',
                          'For the three months ended April 30, 2023 Canadian Banking',
                          'For the three months ended April 30, 2023 International Banking',
                          'For the three months ended April 30, 2023 Global Wealth Management',
                          'For the three months ended April 30, 2023 Global Banking and Markets',
                          'For the three months ended April 30, 2023 Other',
                          'For the three months ended April 30, 2023 Total',
                          ]

          result_table.append(newDF)





<ipython-input-364-832ad82f8711>:87: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.isna(row[0]):
<ipython-input-364-832ad82f8711>:92: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  found = text in str(row[0])
<ipython-input-364-832ad82f8711>:89: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  found = any(text in str(row[col]) for col in check_columns)
<ipython-input-565-f18265f817cc>:102: SettingWithCopyWarning: 
A value is tryin

In [566]:
print(f'Total Tables: {len(result_table)}\n')


#iterate tables and apply different cleaning techniques depending on the table
for index, table in enumerate(result_table):
  print(f'\n Table number: {index +1}')
  print(f'Table dimmensions: {table.shape}' )



Total Tables: 10


 Table number: 1
Table dimmensions: (15, 6)

 Table number: 2
Table dimmensions: (38, 6)

 Table number: 3
Table dimmensions: (14, 7)

 Table number: 4
Table dimmensions: (14, 7)

 Table number: 5
Table dimmensions: (14, 7)

 Table number: 6
Table dimmensions: (14, 7)

 Table number: 7
Table dimmensions: (16, 7)

 Table number: 8
Table dimmensions: (11, 10)

 Table number: 9
Table dimmensions: (10, 10)

 Table number: 10
Table dimmensions: (5, 13)


In [567]:
#save all tables for future use
for index, table in enumerate(result_table):
  table.to_csv(f'table_{index}.csv', index=False)

### 6. Exploratory Data Analysis

### 7. Modeling

### 8. Evaluation Metrics

### 9. Testing

### 10. Conclusion

In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requ

In [3]:
from datasets import load_dataset

# Load the English-Spanish dataset from OPUS
dataset = load_dataset("opus_books", "en-es")

# Access the training data
train_data = dataset["train"]

# Preview the dataset



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/93470 [00:00<?, ? examples/s]

{'id': '0', 'translation': {'en': 'Source: Project GutenbergAudiobook available here', 'es': 'Source: Wikisource & librodot.com'}}


AttributeError: 'Dataset' object has no attribute 'id'

In [4]:
print(train_data[0:10])

{'id': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 'translation': [{'en': 'Source: Project GutenbergAudiobook available here', 'es': 'Source: Wikisource & librodot.com'}, {'en': 'Sense and Sensibility', 'es': 'SENTIDO Y SENSIBILIDAD'}, {'en': 'Jane Austen', 'es': 'JANE AUSTEN'}, {'en': 'CHAPTER 1', 'es': 'CAPITULO I'}, {'en': 'The family of Dashwood had long been settled in Sussex.', 'es': 'La familia Dashwood llevaba largo tiempo afincada en Sussex.'}, {'en': 'Their estate was large, and their residence was at Norland Park, in the centre of their property, where, for many generations, they had lived in so respectable a manner as to engage the general good opinion of their surrounding acquaintance.', 'es': 'Su propiedad era de buen tamaño, y en el centro de ella se encontraba la residencia, Norland Park, donde la manera tan digna en que habían vivido por muchas generaciones llegó a granjearles el respeto de todos los conocidos del lugar.'}, {'en': 'The late owner of this estate

In [6]:
import pandas as pd

# Extract texts
source_texts = [item['translation']['en'] for item in dataset['train']]
target_texts = [item['translation']['es'] for item in dataset['train']]

# Create DataFrame
df = pd.DataFrame({'source': source_texts, 'target': target_texts})

# Preview the DataFrame
print(df.head())

                                              source  \
0  Source: Project GutenbergAudiobook available here   
1                              Sense and Sensibility   
2                                        Jane Austen   
3                                          CHAPTER 1   
4  The family of Dashwood had long been settled i...   

                                              target  
0                  Source: Wikisource & librodot.com  
1                             SENTIDO Y SENSIBILIDAD  
2                                        JANE AUSTEN  
3                                         CAPITULO I  
4  La familia Dashwood llevaba largo tiempo afinc...  


In [13]:
df[df['source'].str.contains('financial', na = False)]


,source,target
49658,"What was most unpleasant was that a financial consideration would now be mixed up with the impending reconciliation. The idea that he might be biased by that consideration, might seek a reconciliation in order to sell the forest, offended him.","Leído el correo, Esteban Arkadievich tomó los documentos de la oficina, hojeó con rapidez un par de expedientes, hizo unas observaciones en los márgenes con un enorme lápiz, y luego comenzó a tomarse el café, a la vez que leía el periódico de la mañana, húmeda aún la tinta de imprenta."
60645,"In this way they chattered unceasingly about Karenin, blaming him and laughing at him, while he, barring the way to the member of the State Council whom he had buttonholed, and not pausing for a moment for fear he might slip away, expounded point by point some financial project of his.","Mientras así hablaban de Karenin, criticándole y burlándose de él, éste, cerrando el paso al miembro del Consejo Imperial de quien se había apoderado, no interrumpía ni por un momento la explicación de su proyecto financiero a fin de que no pudiese marcharse."
62689,"Besides, he knew that the twenty roubles she would have had to pay for the journey were of importance to her, and he felt her distressing financial embarrassments as if they had been his own.","Sabía, además, que los veinte rublos que pedían a su cuñada por los caballos constituían para ella una pesada carga, dada su difícil situación económica. La comitiva era muy abigarrada y nada brillante, pero Daria llegaría así con seguridad absoluta, fácilmente y dentro del mismo día, a la propiedad de Vronsky."
69023,"Captain Nemo must have spent millions in acquiring these different specimens, and I was wondering what financial resources he tapped to satisfy his collector's fancies, when these words interrupted me:","El capitán Nemo había debido gastar millones para adquirir tales especímenes. Estaba preguntándome yo cuál sería el alcance de una fortuna que permitía satisfacer tales caprichos de coleccionista, cuando el capitán interrumpió el curso de mi pensamiento."
71920,"I spotted Jidda, an important financial center for Egypt, Syria, Turkey, and the East Indies.","Pude ver Yidda, importante factoría comercial para Egipto, Siria, Turquía y la India."
88232,"Circulation was impeded, and everywhere disputes, discussions, and financial transactions were going on.","La circulación estaba impedida. Se discutía, se disputaba, se voceaba la cotización de Phileas Fogg, como la de los fondos ingleses."


In [5]:
from datasets import load_dataset

# Example of a finance-related dataset
dataset = load_dataset("financial_phrasebank", split='train')

# Preview the dataset
print(dataset[0])

The repository for financial_phrasebank contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/financial_phrasebank.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


ValueError: Config name is missing.
Please pick one among the available configs: ['sentences_allagree', 'sentences_75agree', 'sentences_66agree', 'sentences_50agree']
Example of usage:
	`load_dataset('financial_phrasebank', 'sentences_allagree')`

In [ ]:
print(dataset[0])